In [4]:
#pip install spark-nlp NOT USED
#pip install pyspark NOT USED


#pip install --upgrade pymupdf
#pip install -U sentence-transformers

import pymupdf #for reading pdf files
from sentence_transformers import SentenceTransformer #for getting vectors from text
from pathlib import Path #for reading all pdfs in the folder

def pdf_reader(file_path): #test method for reading pdfs
    #open file
    file = pymupdf.open(file_path)
    
    #read the file
    #out = open("output.txt", "wb") #prints text to output file in the directory
    out = ""
    for page in file: #iterate the pages
        text = page.get_text() #get plain text
        out += text #write text of page
    #print(out)
        
    file.close()
    return out

def txt_reader(): #test method for reading text files
    #open file
    file = open("testerfile.txt")
    
    #read the file
    content = file.read()
    print(content)
    
    #close the file
    file.close()
    return

In [5]:
def main():
    #doc = pymupdf.open("")
    #txt_reader()
    #print(pdf_reader("Merged March 2023 (wine).pdf"))
    return

if __name__ == "__main__":
    main()

In [6]:
# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

sentence1 = "The weather is lovely today."
sentence2 = "It's so sunny outside!"
sentence3 = "He drove to the stadium."

# The sentences to encode
sentences = [
    sentence1,
    sentence2,
    sentence3,
]

# 2. Calculate embeddings by calling model.encode()
embeddings = model.encode(sentences)
print(embeddings.shape)
# [3, 384]
#embedding1 = embeddings[0] #print the embedding for sentence1 in the index 0
#print(embedding1)

# 3. Calculate the embedding similarities
similarities = model.similarity(embeddings, embeddings)
print(similarities)
# tensor([[1.0000, 0.6660, 0.1046],
#         [0.6660, 1.0000, 0.1411],
#         [0.1046, 0.1411, 1.0000]])

(3, 384)
tensor([[1.0000, 0.6660, 0.1046],
        [0.6660, 1.0000, 0.1411],
        [0.1046, 0.1411, 1.0000]])


In [7]:
#logic for ingesting the menus and adding them to a vector space

# returns all file paths that has .pdf as extension in the specified directory, from https://stackoverflow.com/questions/70848337/how-to-read-multiple-pdf-from-a-folder-one-by-one
pdf_search = Path(str(Path.cwd())).rglob("*.pdf") #gets all the pdf files in the current directory
pdf_files = [str(file.absolute()) for file in pdf_search] #iterates over all the pdf files in the current directory and adds them to an array.

pdf_filecontent = [pdf_reader(file) for file in pdf_files] #take the file names and extract the text content from them.

menuembeddings = model.encode(pdf_filecontent)
print(menuembeddings.shape)
menuembedding1 = menuembeddings[2] #using the wine restuarant to tests similarity scores


prompt0 = "I like wine and different alcoholic drinks like pinot and red wine, cabernet sauvignon, rose wine, and sauvignon blanc, any wine or beer is good."
prompt1 = "I want to eat cheeseburgers and sandwiches, they're some of my favorite foods to eat."
prompt2 = "the Dodgers are the best baseball team to ever exist, I like to watch their games on sundays."
prompts = [prompt0, prompt1, prompt2]
promptembeddings = model.encode(prompts)

similarities = []
for i, embedding in enumerate(promptembeddings):
    similarity = model.similarity(embedding, menuembedding1) #compare each prompt in the prompt array to the wine restuarant.
    similarities.append(similarity.item()) #append the similarity score to the array in a scalar format
    
print(similarities)


(4, 384)
[0.5277917385101318, 0.13504096865653992, 0.049987804144620895]
